In [1]:
!pip install Arabic-Stopwords

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 353 kB 4.8 MB/s 
     |████████████████████████████████| 126 kB 28.5 MB/s 


In [2]:
from sklearn.metrics import classification_report ,confusion_matrix ,accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer ,TfidfVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from nltk.tokenize import TreebankWordTokenizer
from keras.preprocessing.text import Tokenizer
import arabicstopwords.arabicstopwords as stp
from keras.models import Sequential
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from keras import layers
import seaborn as sns
import pandas as pd
import collections
import numpy as np
import matplotlib
import argparse
import codecs
import string
import pickle
import nltk
import math 
import sys
import re
import os
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
root_path = '/content/drive/MyDrive/Colab Notebooks/Tasks/questions classification by topic'
data_path = '/content/drive/MyDrive/Colab Notebooks/Tasks/questions classification by topic/DataSets'
results_path = '/content/drive/MyDrive/Colab Notebooks/Tasks/questions classification by topic/result_with_deep_learning'

# Dataset Preparation (pre-processing)

In [ ]:
# data = pd.DataFrame(pd.read_excel(os.path.join(data_path,"IslamWebAR.xlsx"))).to_csv(os.path.join(data_path,"IslamWebAR.csv"),index=False) # IslamWebAR # BinBaz

In [ ]:
data = pd.read_csv(os.path.join(data_path,"IslamWebAR.csv"))

In [ ]:
old_data = pd.read_csv(os.path.join(data_path,"Copy of FullArabicData.csv"))

In [ ]:
print(data.shape)
data.head()

(16839, 5)


,Title,Date,Question,Answer,Detailed Topic Tree
0,وجود من يحل ما حرم الله لا يعني عدم حفظ الدين,2-8-2022,أعرف أنه لا يمكن تحريف القرآن الكريم، فهل يمكن...,الحمد لله والصلاة والسلام على رسول الله وعلى آ...,العقيدة الإسلامية
1,الاعتقادات الأساسية المتعين على كل مسلم الإيما...,17-4-2022,أريد أن أعرف ما يكفيني في العقيدة؛ لأعتقده، بح...,الحمد لله والصلاة والسلام على رسول الله وعلى آ...,العقيدة الإسلامية
2,التلقي والنظر والاتباع يؤدي إلى التوافق ويمنع ...,1-10-2021,مرحبا أيها السادة.عندما يصل الشخص إلى مرحلة ال...,الحمد لله، والصلاة والسلام على رسول الله، وعلى...,العقيدة الإسلامية
3,كتب مختصرة مفيدة في العقيدة,1-8-2018,أريد منكم أن تحيلوني على فتاوى، أو صفحات على م...,الحمد لله والصلاة والسلام على رسول الله وعلى آ...,العقيدة الإسلامية
4,الأعمال التي تدخل في مرتبة الإسلام,29-7-2018,سمعت أن الإسلام ثلاث مراتب هي: الإسلام. الإيما...,الحمد لله، والصلاة والسلام على رسول الله، وعلى...,العقيدة الإسلامية


In [ ]:
print(old_data.shape)
old_data.head()

(26962, 8)


,Question,Answer,Summary,Main Topic,Detailed Topic Tree,Contents,Date,Views
0,"حديث النبي عليه الصلاة والسلام ""أنا بريء من كل...",الحمد لله.أولا : الحديث الذي سألت عنه هو : عَن...,None,الفقه وأصوله,الفقه وأصوله>الفقه>عبادات>الجهاد والهجرة>أحكام...,None,2015-05-17,133529
1,ورد في الحديث أنّ النبي صلى الله عليه وسلم قال...,الحمد لله.أولاً :الإسلام دين الرحمة والسلام كم...,None,الفقه وأصوله,الفقه وأصوله>الفقه>عبادات>الجهاد والهجرة>أحكام...,None,2015-02-28,140479
2,قصتي هي كالتالي : أنا فتاة عمري 25 عاماً ، من ...,الحمد لله.أولاً:نسأل الله أن يرفع عنكم المحنة ...,None,الفقه وأصوله,الفقه وأصوله>الفقه>عبادات>الجهاد والهجرة>أحكام...,None,2012-06-24,21167
3,ما حكم من يموت دفاعاً عن وطنه من غير المسلمين ...,الحمد لله.من قتل في سبيل الله فهو شهيد ومعنى (...,None,الفقه وأصوله,الفقه وأصوله>الفقه>عبادات>الجهاد والهجرة>أحكام...,None,2003-04-14,152557
4,ما هو رأي الإسلام في الدفاع عن النفس؟ أهو من ...,الحمد لله.حفظ النفس والعرض والعقل والمال والدي...,None,الفقه وأصوله,الفقه وأصوله>الفقه>عبادات>الجهاد والهجرة>أحكام...,None,2002-01-29,36482


In [ ]:
old_data.drop(['Answer','Summary','Contents','Date','Views','Detailed Topic Tree'],axis=1,inplace=True)
data.drop(['Answer','Date','Title',],axis=1,inplace=True)

In [ ]:
data.head()

,Question,Detailed Topic Tree
0,أعرف أنه لا يمكن تحريف القرآن الكريم، فهل يمكن...,العقيدة الإسلامية
1,أريد أن أعرف ما يكفيني في العقيدة؛ لأعتقده، بح...,العقيدة الإسلامية
2,مرحبا أيها السادة.عندما يصل الشخص إلى مرحلة ال...,العقيدة الإسلامية
3,أريد منكم أن تحيلوني على فتاوى، أو صفحات على م...,العقيدة الإسلامية
4,سمعت أن الإسلام ثلاث مراتب هي: الإسلام. الإيما...,العقيدة الإسلامية


In [ ]:
old_data.head()

,Question,Main Topic
0,"حديث النبي عليه الصلاة والسلام ""أنا بريء من كل...",الفقه وأصوله
1,ورد في الحديث أنّ النبي صلى الله عليه وسلم قال...,الفقه وأصوله
2,قصتي هي كالتالي : أنا فتاة عمري 25 عاماً ، من ...,الفقه وأصوله
3,ما حكم من يموت دفاعاً عن وطنه من غير المسلمين ...,الفقه وأصوله
4,ما هو رأي الإسلام في الدفاع عن النفس؟ أهو من ...,الفقه وأصوله


In [ ]:
elements_count = collections.Counter(data['Detailed Topic Tree'].tolist())
for key, value in elements_count.items():
    print(f"{key}: {value}")

العقيدة الإسلامية: 50
القرآن الكريم: 25
القرآن الكريم>شبهات حول القرآن الكريم: 179
العقيدة الإسلامية>أركان الإيمان>الإيمان بالله>توحيد الأسماء والصفات>قواعد في الأسماء والصفات: 2
القرآن الكريم>من علوم القرآن الكريم>الناسخ المنسوخ>معناهما وأهميتهما والحكمة منها: 2
القرآن الكريم>قراءات القرآن>القراءات وأقسامها وفوائدها: 1
العقيدة الإسلامية>أركان الإيمان>الإيمان بالكتب>القرآن كلام الله: 3
العقيدة الإسلامية>أركان الإيمان>الإيمان بالكتب>حفظ القرآن من التحريف: 1
العقيدة الإسلامية>أركان الإيمان>الإيمان بالكتب>شبهات حول الإيمان بالكتب: 1
العقيدة الإسلامية>أركان الإيمان>الإيمان بالله>شبهات حول توحيد الله: 5
العقيدة الإسلامية>أركان الإيمان>الإيمان باليوم الآخر>علامات الساعة الكبرى: 1
القرآن الكريم>جمع القرآن الكريم: 1
القرآن الكريم>مختارات من تفسير الآيات: 1705
العقيدة الإسلامية>أديان وفرق ومذاهب>أديان>اليهودية: 1
العقيدة الإسلامية>أركان الإيمان>الإيمان بالله>توحيد الأسماء والصفات>من الأسماء الحسنى: 1
العقيدة الإسلامية>أركان الإيمان>الإيمان بالقدر>شبهات حول الإيمان بالقدر: 3
العقيدة الإسلامية>أر

In [ ]:
print(len(data["Detailed Topic Tree"].tolist()))
label = list(set(data['Detailed Topic Tree'].tolist()))
len(label)

16809


175

In [ ]:
print(len(old_data["Main Topic"].tolist()))
list_of_label = list(set(old_data['Main Topic'].tolist()))
len(list_of_label)

26962


11

In [ ]:
print(list_of_label)

['الحديث وعلومه', 'السياسة الشرعية', 'العقيدة', 'التربية', 'العلم والدعوة', 'القرآن وعلومه', 'فقه الأسرة', 'التاريخ والسيرة', 'مشكلات نفسية واجتماعية', 'الفقه وأصوله', 'الآداب والأخلاق والرقائق']


In [ ]:
print(label)

['فقه العبادات', 'الأطعمة والأشربة والصيد>الحيوان وما يتعلق به', 'الفضائل والتراجم>فضائل الأيام والأوقات والشهور', 'فقه المعاملات>البيع>أركان البيع وشروطه', 'العقيدة الإسلامية>البدع والمحدثات>منهج السلف في التعامل مع المبتدعة', 'فقه العبادات>الصلاة>صفة الصلاة>السجود والرفع منه', 'العقيدة الإسلامية>الصحابة وأهل البيت>مراتب الصحابة وتفاضلهم', 'القرآن الكريم>شخصيات قرآنية', 'فقه المعاملات>قضايا مالية معاصرة>التأمين', 'فقه الأسرة المسلمة>العدة والمتعة>عدة الطلاق', 'الآداب والأخلاق والرقائق>الأخلاق', 'فقه الأسرة المسلمة', 'القرآن الكريم>مقدمات في القرآن>فضل وآداب تلاوة القرآن وتعلمه', 'العقيدة الإسلامية>الإيمان ونواقضه>الشرك>الشرك الأصغر', 'العقيدة الإسلامية>الإيمان ونواقضه>الشرك>صور من الشرك', 'الأذكار والأدعية>أذكار اليوم والليلة>أذكار أخرى', 'العقيدة الإسلامية>مقدمات عامة>مصادر علم العقيدة', 'الحديث الشريف>علوم الحديث>أنواع الحديث', 'العقيدة الإسلامية>أركان الإيمان>الإيمان باليوم الآخر>الموت وخروج الروح', 'طب وإعلام وقضايا معاصرة>وسائل ترفيه>وسائل أخرى', 'القرآن الكريم>شبهات حول القرآن ا

In [ ]:
def isNaN(string):
    if string == string:
        return string
    return "nan"

In [ ]:
data['Detailed Topic Tree'] = data['Detailed Topic Tree'].apply(isNaN)
data = data[data['Detailed Topic Tree']!='nan']
data[data['Detailed Topic Tree']=='nan'].shape

(0, 2)

In [ ]:
def get_main_topic(x):
  x = str(x)
  if '>' in x:
    x = x[:x.index('>')]
  return x

In [ ]:
elements_count = collections.Counter(data['Detailed Topic Tree'].tolist())
for key, value in elements_count.items():
    print(f"{key}: {value}")

العقيدة الإسلامية: 50
القرآن الكريم: 25
القرآن الكريم>شبهات حول القرآن الكريم: 179
العقيدة الإسلامية>أركان الإيمان>الإيمان بالله>توحيد الأسماء والصفات>قواعد في الأسماء والصفات: 2
القرآن الكريم>من علوم القرآن الكريم>الناسخ المنسوخ>معناهما وأهميتهما والحكمة منها: 2
القرآن الكريم>قراءات القرآن>القراءات وأقسامها وفوائدها: 1
العقيدة الإسلامية>أركان الإيمان>الإيمان بالكتب>القرآن كلام الله: 3
العقيدة الإسلامية>أركان الإيمان>الإيمان بالكتب>حفظ القرآن من التحريف: 1
العقيدة الإسلامية>أركان الإيمان>الإيمان بالكتب>شبهات حول الإيمان بالكتب: 1
العقيدة الإسلامية>أركان الإيمان>الإيمان بالله>شبهات حول توحيد الله: 5
العقيدة الإسلامية>أركان الإيمان>الإيمان باليوم الآخر>علامات الساعة الكبرى: 1
القرآن الكريم>جمع القرآن الكريم: 1
القرآن الكريم>مختارات من تفسير الآيات: 1705
العقيدة الإسلامية>أديان وفرق ومذاهب>أديان>اليهودية: 1
العقيدة الإسلامية>أركان الإيمان>الإيمان بالله>توحيد الأسماء والصفات>من الأسماء الحسنى: 1
العقيدة الإسلامية>أركان الإيمان>الإيمان بالقدر>شبهات حول الإيمان بالقدر: 3
العقيدة الإسلامية>أر

In [ ]:
data['Main Topic'] = data['Detailed Topic Tree'].apply(get_main_topic)

In [ ]:
data.drop(['Detailed Topic Tree'],axis=1,inplace=True)

In [ ]:
elements_count = collections.Counter(data['Main Topic'].tolist())
for key, value in elements_count.items():
    print(f"{key}: {value}")

العقيدة الإسلامية: 106
القرآن الكريم: 1919
الحديث الشريف: 39
السيرة النبوية: 38
الدعوة ووسائلها: 1718
طب وإعلام وقضايا معاصرة: 26
فكر وسياسة وفن: 70
فقه المعاملات: 48
الفضائل والتراجم: 364
الآداب والأخلاق والرقائق: 8744
الأذكار والأدعية: 496
فقه العبادات: 10
فقه الأسرة المسلمة: 14
فقه المواريث: 1616
فقه الجنايات: 4
الحدود والتعزيرات: 10
الأطعمة والأشربة والصيد: 342
الأقضية والشهادات: 267
الأيمان والنذور: 1
اللباس والزينة: 2
أخبار: 3
تراجم وشخصيات: 381
أصول الفقه وقواعده: 277
مصادر الفقه الإسلامي: 314


In [ ]:
print(len(data['Main Topic'].tolist()))
label = list(set(data['Main Topic'].tolist()))
len(label)

16809


24

In [ ]:
for i in label:
  print(i)

فقه العبادات
الحديث الشريف
الأطعمة والأشربة والصيد
أصول الفقه وقواعده
الآداب والأخلاق والرقائق
الحدود والتعزيرات
السيرة النبوية
طب وإعلام وقضايا معاصرة
فقه الأسرة المسلمة
أخبار
فقه المواريث
الأيمان والنذور
فقه الجنايات
تراجم وشخصيات
العقيدة الإسلامية
فقه المعاملات
مصادر الفقه الإسلامي
اللباس والزينة
الأذكار والأدعية
الدعوة ووسائلها
الفضائل والتراجم
فكر وسياسة وفن
القرآن الكريم
الأقضية والشهادات


In [ ]:
from sklearn.utils import shuffle
frames = [data, old_data]
df = pd.concat(frames)
df = shuffle(df)
df.to_csv(os.path.join(data_path,'IslamWebAR&FullArabicData.csv'), index=False) 

In [ ]:
# frames = []
# for label in list_of_label:
#   x = data[data['Main Topic']==label][:20]
#   frames.append(x)
# result = pd.concat(frames)
# print(result.shape)
# p = os.path.join(data_path,'get sample from all class')
# result.to_csv(os.path.join(p,'Sample of IslamWebAR.csv'), index=False) 
# result.head(25)

In [ ]:
def isNaN(string):
    if string == string:
        return string
    return "nan"

In [ ]:
data['Question'] = data['Question'].apply(isNaN)
data[data['Question']=='nan'].shape

(30, 2)

In [ ]:
arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

In [ ]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

In [ ]:
def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text

In [ ]:
def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [ ]:
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [ ]:
def clean_text(text):
    text = remove_diacritics(text)
    text = remove_punctuations(text)
    text = remove_repeating_char(text)
    text = re.sub(r"([@A-Za-z0-9_ـــــــــــــ]+)|[^\w\s]|#|http\S+", " ", text)
    text = " ".join([word for word in text.split() if not stp.is_stop(u"{}".format(word))])
    text = " ".join([word for word in text.split() if len(word)>2])
    return text

In [ ]:
data['Question'] = data['Question'].apply(clean_text)

In [ ]:
def marg(text):
    return " ".join([sen for sen in text]) if type(text) is list else text

In [ ]:
x_train_text_label = []
#y_train_text_label = []

x_test_text_label = []
y_test_text_label = []

for label in list_of_label:
    Temp = data[data['Main Topic']==label]['Question'].tolist()
    x_test_text_label.append(Temp[:int(math.floor(len(Temp)*0.10))])
    x_train_text_label.append(marg(Temp[int(math.ceil(len(Temp)*0.10)):]))
    
    Temp = data[data['Main Topic']==label]['Main Topic'].tolist()
    y_test_text_label.append(Temp[:int(math.floor(len(Temp)*0.10))])
    #y_train_text_label.append(marg(Temp[int(math.ceil(len(Temp)*0.10)):]))
# x_test_text_label = sum(x_test_text_label, [])
x_test_text_label = list(np.concatenate(x_test_text_label).flat)

# y_test_text_label = sum(y_test_text_label, [])
y_test_text_label = list(np.concatenate(y_test_text_label).flat)

In [ ]:
elements_count = collections.Counter(y_test_text_label)
for key, value in elements_count.items():
    print(f"{key}: {value}")

الفقه وأصوله: 1447
فقه الأسرة: 314
العقيدة: 264
الآداب والأخلاق والرقائق: 249
الحديث وعلومه: 163
القرآن وعلومه: 89
العلم والدعوة: 69
التاريخ والسيرة: 49
مشكلات نفسية واجتماعية: 35
التربية: 9
السياسة الشرعية: 4


In [ ]:
def tfidf(data):
    tfidf_vectorizer = TfidfVectorizer(min_df=0.3, max_df=0.7, ngram_range=(1, 2))
    train = tfidf_vectorizer.fit_transform(data)
    return train, tfidf_vectorizer

In [ ]:
def cv(data): # Bag of words (BOW)  , binary=True
    count_vectorizer = CountVectorizer()
    emb = count_vectorizer.fit_transform(data)
    return emb, count_vectorizer

In [5]:
data = pd.read_csv(os.path.join(data_path,'IslamWebAR&FullArabicData.csv'))

In [6]:
print(data.shape)
data.head()

(43771, 2)


,Question,Main Topic
0,هل هذا حديث يجوز الاستشهاد به قال رسول الله صل...,الحديث وعلومه
1,ما صحة قراءة خمس آيات صباحا ومساء فيها خمسون ق...,الأذكار والأدعية
2,مات أبي ولم يكن له أبناء غيري (أنا أنثى بالغة)...,فقه المواريث
3,أنا طالبة في الثانوية العامة، أنعم عليّ الله س...,الآداب والأخلاق والرقائق
4,بسم الله الرحمن الرحيمالسؤال/ من ماذا خلق الله...,الأطعمة والأشربة والصيد


In [8]:
data['Main Topic'].value_counts()

الفقه وأصوله                14471
الآداب والأخلاق والرقائق    11238
فقه الأسرة                   3145
العقيدة                      2645
القرآن الكريم                1919
الدعوة ووسائلها              1718
الحديث وعلومه                1631
فقه المواريث                 1616
القرآن وعلومه                 899
العلم والدعوة                 695
التاريخ والسيرة               496
الأذكار والأدعية              496
تراجم وشخصيات                 381
الفضائل والتراجم              364
مشكلات نفسية واجتماعية        351
الأطعمة والأشربة والصيد       342
مصادر الفقه الإسلامي          314
أصول الفقه وقواعده            277
الأقضية والشهادات             267
العقيدة الإسلامية             106
التربية                        93
فكر وسياسة وفن                 70
فقه المعاملات                  48
السياسة الشرعية                42
الحديث الشريف                  39
السيرة النبوية                 38
طب وإعلام وقضايا معاصرة        26
فقه الأسرة المسلمة             14
الحدود والتعزيرات              10
فقه العبادات  

In [9]:
print(len(data['Main Topic'].tolist()))
list_of_label = list(set(data['Main Topic'].tolist()))
len(list_of_label) 

43771


34

In [11]:
ALFUQH_W_FROAH = ['فقه الجنايات','فقه العبادات','فقه الأسرة','أصول الفقه وقواعده','فقه الأسرة المسلمة','مصادر الفقه الإسلامي','فقه المعاملات','فقه المواريث','الفقه وأصوله','اللباس والزينة','الأيمان والنذور','فقه','الأقضية والشهادات','الحدود والتعزيرات'] # الفقه وفروعه
ALTARAGEM_W_AHAKHBAR = ['تراجم وشخصيات','الفضائل والتراجم','أخبار'] # التراجم والأخبار
ALHADETH_AHSHAREEF = ['الحديث الشريف','الحديث وعلومه'] # الحديث الشريف
MUSHKELAT_IGTEMAEA_W_FEKREA = ['مشكلات نفسية واجتماعية','فكر وسياسة وفن'] # مشكلات اجتماعية وفكرية
Alquran_w_ulomah = ['القرآن الكريم','القرآن وعلومه'] # القرآن و علومه
Alaqedah_Alesalmea = ['العقيدة الإسلامية','العقيدة'] # العقيدة الإسلامية
Altarekh_w_Alsunah = ['السيرة النبوية','التاريخ والسيرة'] # التاريخ و السيرة النبوية
# Fukh = ['الأقضية والشهادات','الحدود والتعزيرات'] # فقه
Altarbeh_w_Alaadab  = ['التربية','الآداب والأخلاق والرقائق'] # التربية و الآداب
Mushkelat_Agtemaya_w_Feqreah = ['طب وإعلام وقضايا معاصرة'] # مشكلات اجتماعية وفكرية
Alelm_w_Aldawa = ['العلم والدعوة','الدعوة ووسائلها'] # العلم و الدعوة 

# x = []
# for la in list_of_label:
#   if la not in ALFUQH_W_FROAH+ALTARAGEM_W_AHAKHBAR+ALHADETH_AHSHAREEF+MUSHKELAT_IGTEMAEA_W_FEKREA+Alquran_w_ulomah+Alaqedah_Alesalmea+Altarekh_w_Alsunah+Fukh+Altarbeh_w_Alaadab+Mushkelat_Agtemaya_w_Feqreah+Alelm_w_Aldawa:
#     x.append(la)
# elements_count = collections.Counter(x)
# for key, value in elements_count.items():
#     print(f"{key}: {value}")

In [12]:
## القران الكريم - القران وعلومه ->القرآن وعلومه
## التاريخ والسيرة - السيرة النبوية ->التاريخ والسيرة النبوية
## العقيدة - العقيدة الاسلامية -> العقيدة الإسلامية
## اقضية وشهادات - حدود وتعزيزات ->فقه ******
## الاداب والاخلاق - التربية->التربية والآداب
# طب وإعلام وقضايا معاصرة ->مشكلات اجتماعية وفكرية
# العلم والدعوة - الدعوة ووسائلها ->العلم والدعوة

# السياسة الشرعية لحالها

In [13]:
def Filter(x):
  if x in ALFUQH_W_FROAH:
    return 'الفقه وفروعه'
  elif x in ALTARAGEM_W_AHAKHBAR:
    return 'التراجم والأخبار'
  elif x in ALHADETH_AHSHAREEF:
    return 'الحديث الشريف'
  elif x in MUSHKELAT_IGTEMAEA_W_FEKREA:
    return 'مشكلات اجتماعية وفكرية'
  elif x in Alquran_w_ulomah:
    return 'القرآن و علومه'
  elif x in Alaqedah_Alesalmea:
    return 'العقيدة الإسلامية'
  elif x in Altarekh_w_Alsunah:
    return 'التاريخ و السيرة النبوية'
  # elif x in Fukh:
  #   return 'فقه'
  elif x in Altarbeh_w_Alaadab:
    return 'التربية و الآداب'
  elif x in Mushkelat_Agtemaya_w_Feqreah:
    return 'مشكلات اجتماعية وفكرية'
  elif x in Alelm_w_Aldawa:
    return 'العلم و الدعوة'
  return x

In [14]:
# 'فقه',
classes = ['الفقه وفروعه','التراجم والأخبار','الحديث الشريف','مشكلات اجتماعية وفكرية','القرآن و علومه','العقيدة الإسلامية','التاريخ و السيرة النبوية','التربية و الآداب','العلم و الدعوة']
len(classes)

9

In [15]:
data['Main Topic'] = data['Main Topic'].apply(Filter)

In [16]:
class_0=[]
class_1=[]
class_2=[]
class_3=[]
class_4=[]
class_5=[]
class_6=[]
class_7=[]
class_8=[]
class_9=[]
# class_10=[]

for d in data['Main Topic'].tolist():
  if d == classes[0]:
    class_0.append(d)
  elif d == classes[1]:
    class_1.append(d)
  elif d == classes[2]:
    class_2.append(d)
  elif d == classes[3]:
    class_3.append(d)
  elif d == classes[4]:
    class_4.append(d)
  elif d == classes[5]:
    class_5.append(d)
  elif d == classes[6]:
    class_6.append(d)
  elif d == classes[7]:
    class_7.append(d)
  elif d == classes[8]:
    class_8.append(d)
  # elif d == classes[9]:
  #   class_9.append(d)
  else:
    class_9.append(d)
print("=====================================================")
for c in [class_0,class_1,class_2,class_3,class_4,class_5,class_6,class_7,class_8]: # ,class_9
  elements_count = collections.Counter(c)
  for key, value in elements_count.items():
      print(f"{key}: {value}")
print("=====================================================")
elements_count = collections.Counter(class_9) # class_10
for key, value in elements_count.items():
    print(f"{key}: {value}")


الفقه وفروعه: 20179
التراجم والأخبار: 748
الحديث الشريف: 1670
مشكلات اجتماعية وفكرية: 447
القرآن و علومه: 2818
العقيدة الإسلامية: 2751
التاريخ و السيرة النبوية: 534
التربية و الآداب: 11331
العلم و الدعوة: 2413
الأذكار والأدعية: 496
الأطعمة والأشربة والصيد: 342
السياسة الشرعية: 42


In [17]:
data['Main Topic'].value_counts()

الفقه وفروعه                20179
التربية و الآداب            11331
القرآن و علومه               2818
العقيدة الإسلامية            2751
العلم و الدعوة               2413
الحديث الشريف                1670
التراجم والأخبار              748
التاريخ و السيرة النبوية      534
الأذكار والأدعية              496
مشكلات اجتماعية وفكرية        447
الأطعمة والأشربة والصيد       342
السياسة الشرعية                42
Name: Main Topic, dtype: int64

In [19]:
# data.to_csv(os.path.join(data_path,'after_collection_IslamWebAR&FullArabicData_v2.csv'),index=False)

In [20]:
df = pd.read_csv(os.path.join(data_path,'after_collection_IslamWebAR&FullArabicData_v2.csv'))
df.head()

,Question,Main Topic
0,هل هذا حديث يجوز الاستشهاد به قال رسول الله صل...,الحديث الشريف
1,ما صحة قراءة خمس آيات صباحا ومساء فيها خمسون ق...,الأذكار والأدعية
2,مات أبي ولم يكن له أبناء غيري (أنا أنثى بالغة)...,الفقه وفروعه
3,أنا طالبة في الثانوية العامة، أنعم عليّ الله س...,التربية و الآداب
4,بسم الله الرحمن الرحيمالسؤال/ من ماذا خلق الله...,الأطعمة والأشربة والصيد


In [21]:
df['Main Topic'].value_counts()

الفقه وفروعه                20179
التربية و الآداب            11331
القرآن و علومه               2818
العقيدة الإسلامية            2751
العلم و الدعوة               2413
الحديث الشريف                1670
التراجم والأخبار              748
التاريخ و السيرة النبوية      534
الأذكار والأدعية              496
مشكلات اجتماعية وفكرية        447
الأطعمة والأشربة والصيد       342
السياسة الشرعية                42
Name: Main Topic, dtype: int64